# Relax Inc. Data Science Sample Take Home Challenge

This is a practice take home challenge from Relax Inc.

The data is available as two attached CSV files: takehome_user_engagement. csv takehome_users . csv The data has the following two tables:

1.]   A user table ( "takehome_users" ) with data on 12,000 users who signed up for the product in the last two years. This table includes:
        
- name : the user's name
- object_id : the user's id
- email : email address
- creation_source : how their account was created. This takes on one of 5 values !
- PERSONAL_PROJECTS : invited to join another user's personal workspace
- GUEST_INVITE : invited to an organization as a guest (limited permissions)
- ORG_INVITE : invited to an organization (as a full member)
- SIGNUP : signed up via the website
- SIGNUP_GOOGLE_AUTH : signed up using Google Authentication (using a Google email account for their login id)
- creation_time : when they created their account
- last_session_creation_time : unix timestamp of last login
- opted_in_to_mailing_list : whether they have opted into receiving marketing emails
- enabled_for_marketing_drip : whether they are on the regular marketing email drip
- org_id : the organization (group of users) they belong to
- invited_by_user_id : which user invited them to join (if applicable).
     
2.]   A usage summary table ("takehome_user_engagement") that has a row for each day that a user logged into the product. Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven day period, identify which factors predict future user adoption.
     
We suggest spending 1-2 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('fivethirtyeight')

In [2]:
users = pd.read_csv('takehome_users.csv', encoding='latin-1',parse_dates=True)
engage = pd.read_csv('takehome_user_engagement.csv', parse_dates=True)

In [3]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [6]:
engage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [7]:
#convert the time_stamp to datetime and set it as the index
engage.time_stamp = pd.to_datetime(engage.time_stamp)
engage = engage.set_index('time_stamp', drop= True)

In [8]:
engage.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1


In [9]:
#Group by user_id and resample to 1 week period, sum over period
df_agg = engage.groupby([pd.Grouper(freq='W'),'user_id']).sum()

In [23]:
df_agg.head()

visited
time_stamp user_id         
2012-06-03 563            1
           1693           1
           1995           1
           2120           1
           2136           1

In [ ]:
# users.set_index('object_id').join(engage.set_index('user_id'))

Define an "adopted user" as a user who has logged into the product on three separate days in at least one seven day period, identify which factors predict future user adoption.



In [25]:
#find all user id's with a sum of 3 or more indicating an adopted user
df_adopt = df_agg[df_agg.visited>=3].unstack(level=1).melt()
adopted_users = pd.DataFrame(df_adopt.user_id.unique(),index=range(df_adopt.user_id.unique().shape[0]),columns=['user_id'])

In [44]:
adopted_users.head()

,user_id
0,1693
1,728
2,11764
3,5297
4,6171


In [45]:
#create df of features
df_join = users.merge(adopted_users,how='inner',left_on='object_id',right_on='user_id')
df_join.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,42


In [46]:
# drop irrelevant columns
drop_col = list(df_join.columns[0:4])
drop_col.append('user_id')
df_join = df_join.drop(drop_col,axis=1)

In [47]:
df_join

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,SIGNUP,1.401364e+09,0,0,58,NaN
3,GUEST_INVITE,1.401518e+09,0,0,401,79.0
4,SIGNUP,1.401045e+09,1,0,235,NaN
...,...,...,...,...,...,...
1440,GUEST_INVITE,1.400657e+09,0,0,65,11251.0
1441,GUEST_INVITE,1.401524e+09,0,0,15,5688.0
1442,GUEST_INVITE,1.401411e+09,1,1,52,6647.0
1443,GUEST_INVITE,1.400757e+09,1,0,31,6410.0


In [48]:
#fill na values in invited_by_user column
df_join['invited_by_user_id'].fillna(value=0,inplace=True)

In [57]:
#one hot encode creation_source feature
df_create = pd.get_dummies(df_join['creation_source'])
df_features = pd.concat([df_join,df_create],axis=1)
df_features.drop('creation_source',axis=1,inplace=True)


In [58]:
#convert columns to float64
for col in df_features.columns:
    df_features[col] = df_features[col].astype('float64')
df_features.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1.396238e+09,0.0,0.0,1.0,316.0,0.0,1.0,0.0,0.0,0.0
1,1.401833e+09,1.0,1.0,318.0,4143.0,0.0,1.0,0.0,0.0,0.0
2,1.401364e+09,0.0,0.0,58.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.401518e+09,0.0,0.0,401.0,79.0,1.0,0.0,0.0,0.0,0.0
4,1.401045e+09,1.0,0.0,235.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#scale data
scaler = StandardScaler()
features = scaler.fit_transform(df_features)

#fit PCA
pca = PCA()
components = pca.fit_transform(features)

In [60]:
np.sum(pca.explained_variance_ratio_[0:7])

0.9227803367525598

The first seven principal components account for about 92 % of the variation in the data. However, we will look at the relative importance of all features in the feature set.

In [62]:
df_comp = pd.DataFrame(pca.components_,columns=df_features.columns,index=['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6','PC-7','PC-8','PC-9','PC-10'])
#absolute values of correlation with principal components
best_features = np.absolute(df_comp[np.absolute(df_comp) > 0.1])
best_features.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
PC-1,NaN,NaN,NaN,NaN,0.637653,0.232628,0.486575,0.233976,0.390324,0.296733
PC-2,NaN,0.695712,0.69891,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PC-3,0.116892,NaN,NaN,NaN,NaN,0.784060,0.590037,NaN,NaN,NaN
PC-4,NaN,NaN,NaN,0.261476,NaN,NaN,NaN,0.108246,0.648711,0.697782
PC-5,0.230407,NaN,NaN,0.219055,NaN,NaN,NaN,0.825688,0.296054,0.350811


In [63]:
#sum correlations to get a relative estimate of the feature importance
best_features.head(7).sum(axis=0).sort_values(ascending=False)

org_id                        1.792770
SIGNUP_GOOGLE_AUTH            1.705164
last_session_creation_time    1.700941
SIGNUP                        1.524752
PERSONAL_PROJECTS             1.438816
ORG_INVITE                    1.076612
GUEST_INVITE                  1.016688
enabled_for_marketing_drip    0.698910
opted_in_to_mailing_list      0.695712
invited_by_user_id            0.637653
dtype: float64

The first seven principal components account for most of the data variation in the dataset of adopted users, meaning that these variables are highly predictive of adopted users. 

From the above summed correlations with the top principal components, we see three features that stand out: Organization id, creation_source and the last session creation time. These factors are the most predictive of future user adoption.